In [4]:
import os
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import chromedriver_autoinstaller

# Cài đặt ChromeDriver nếu chưa có
chromedriver_autoinstaller.install()

# Tên thư mục chứa video
VIDEO_FOLDER = 'C:\\Users\\nguyenmanhcuong1991\\Desktop\\video\\'

# Hàm khởi tạo trình duyệt và đăng nhập vào YouTube
def login_youtube(email, password):
    driver = webdriver.Chrome()
    driver.get("https://www.youtube.com/")

    # Nhấn vào nút đăng nhập
    login_button = driver.find_element(By.XPATH, '//*[@aria-label="Sign in"]')
    login_button.click()

    # Nhập email
    email_input = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@type="email"]'))
    )
    email_input.send_keys(email)
    email_input.send_keys(Keys.RETURN)

    # Nhập password
    password_input = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@type="password"]'))
    )
    password_input.send_keys(password)
    password_input.send_keys(Keys.RETURN)

    # Chờ đến khi đăng nhập hoàn tất
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@aria-label="YouTube Home"]'))
    )

    return driver

# Hàm upload video lên YouTube
def upload_video(driver, file_path, title, description, tags):
    driver.get("https://studio.youtube.com/")

    # Nhấn vào nút upload video
    upload_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@aria-label="Create"]'))
    )
    upload_button.click()

    # Chọn tùy chọn upload video
    upload_video_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@aria-label="Upload videos"]'))
    )
    upload_video_button.click()

    # Upload file video
    file_input = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@type="file"]'))
    )
    file_input.send_keys(file_path)

    # Điền tiêu đề và mô tả
    title_input = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@aria-label="Add a title"]'))
    )
    title_input.send_keys(title)

    description_input = driver.find_element(By.XPATH, '//*[@aria-label="Tell viewers about your video"]')
    description_input.send_keys(description)

    # Thêm tags
    tags_input = driver.find_element(By.XPATH, '//*[@aria-label="Add tags"]')
    tags_input.send_keys(", ".join(tags))

    # Nhấn vào nút tiếp tục và công khai video
    next_button = driver.find_element(By.XPATH, '//*[@aria-label="Next"]')
    next_button.click()

    # Đợi cho đến khi video được upload hoàn tất và lấy URL
    video_url = WebDriverWait(driver, 600).until(
        EC.presence_of_element_located((By.XPATH, '//*[contains(@href, "https://youtu.be/")]'))
    ).get_attribute("href")

    return video_url

# Hàm tạo playlist
def create_playlist(driver, title, description):
    driver.get("https://www.youtube.com/playlist?list=WL")

    # Nhấn vào nút tạo playlist
    create_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@aria-label="New playlist"]'))
    )
    create_button.click()

    # Điền tiêu đề và mô tả playlist
    title_input = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@aria-label="Title"]'))
    )
    title_input.send_keys(title)

    description_input = driver.find_element(By.XPATH, '//*[@aria-label="Description"]')
    description_input.send_keys(description)

    # Nhấn vào nút tạo
    create_button = driver.find_element(By.XPATH, '//*[@aria-label="Create"]')
    create_button.click()

    # Lấy URL của playlist
    playlist_url = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@aria-label="Playlist"]'))
    ).get_attribute("href")

    return playlist_url

# Hàm thêm video vào playlist
def add_video_to_playlist(driver, video_url, playlist_url):
    driver.get(video_url)

    # Nhấn vào nút thêm vào playlist
    add_to_playlist_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@aria-label="Save to playlist"]'))
    )
    add_to_playlist_button.click()

    # Chọn playlist
    playlist_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, f'//*[@title="{playlist_url}"]'))
    )
    playlist_button.click()

def main():
    email = "dichphong9119@gmail.com"
    password = "kiengmotPIDF4231!"

    driver = login_youtube(email, password)

    try:
        for file in os.listdir(VIDEO_FOLDER):
            if file.endswith(".mp4"):
                parts = file.split('-')
                lop = parts[0]
                video_number = parts[1].split('.')[0]

                title = f"{lop}-{video_number}"
                description = f"Description for {title}"
                tags = ["tag1", "tag2"]

                # Upload video
                video_url = upload_video(driver, os.path.join(VIDEO_FOLDER, file), title, description, tags)
                print(f"Uploaded video '{title}' with URL: {video_url}")

                # Create playlist if not exists
                playlist_title = lop
                playlist_description = f"Playlist for {lop}"
                playlist_url = create_playlist(driver, playlist_title, playlist_description)
                print(f"Created playlist '{playlist_title}' with URL: {playlist_url}")

                # Add video to playlist
                add_video_to_playlist(driver, video_url, playlist_url)
                print(f"Added video '{title}' to playlist '{playlist_title}'")

                # Thời gian nghỉ giữa các lần gọi API để tránh bị giới hạn
                time.sleep(60)

    except Exception as e:
        print("An error occurred:", e)
    finally:
        driver.quit()

if __name__ == "__main__":
    main()


ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=125.0.6422.112)
Stacktrace:
	GetHandleVerifier [0x0079B793+45827]
	(No symbol) [0x0072DB74]
	(No symbol) [0x0062138D]
	(No symbol) [0x0065CED9]
	(No symbol) [0x0065B825]
	(No symbol) [0x00682DFC]
	(No symbol) [0x00658075]
	(No symbol) [0x00683094]
	(No symbol) [0x0069C034]
	(No symbol) [0x00682B96]
	(No symbol) [0x00656998]
	(No symbol) [0x0065751D]
	GetHandleVerifier [0x00A543C3+2899763]
	GetHandleVerifier [0x00AA77ED+3240797]
	GetHandleVerifier [0x00821264+593364]
	GetHandleVerifier [0x0082818C+621820]
	(No symbol) [0x00736F54]
	(No symbol) [0x00733658]
	(No symbol) [0x007337F7]
	(No symbol) [0x007258AE]
	BaseThreadInitThunk [0x75E87BA9+25]
	RtlInitializeExceptionChain [0x7713BE3B+107]
	RtlClearBits [0x7713BDBF+191]
